# CUDA Ufuncs

Numba’s vectorize allows Python functions taking scalar input arguments to be used as NumPy ufuncs. Using the `vectorize()` decorator, Numba can compile a pure Python function into a ufunc that operates over NumPy arrays and executes on the GPU.

Using vectorize(), you write your function as operating over input scalars, rather than arrays. Numba will generate the surrounding loop (or kernel) allowing efficient iteration over the actual inputs.

In [1]:
import numba
from numba import vectorize,cuda
import numpy as np
import math

### Law of Cosines

For a triangle with sides $a$, $b$, and $c$ the law of cosines dictates that

$$
\frac{a^2+b^2-c^2}{2ab}=\cos C
$$

### Numba Ufunc Kernel

Below, we define the GPU-accelerated eager, or decoration-time, compilation vectorized function by providing signatures to the decorator and specifying `target='cuda'`.  GPU-targeted Ufuncs require signatures. 

As an exercise, complete the missing lines of code.  You will have to specify the signature(s) and computation.  

In [2]:
@vectorize(# --- FILL THIS IN ---,
           target='cuda')
def compute_angle(a, b, c):
    # --- FILL THIS IN ---
    

### Prepare Data

In [3]:
N = int(5e8)
dtype = np.float32

# prepare the input
a = np.array(np.random.sample(N)+3, dtype=dtype)
b = np.array(np.random.sample(N)+4, dtype=dtype)
c = np.array(np.random.sample(N)+5, dtype=dtype)

### Call GPU Ufunc
Calling a GPU Ufunc is as straight forward as calling a numpy function.  Under the hood, the CUDA launch configuration is handled automatically.

In [4]:
%%timeit -n2 -r5 -o
C_GPU = compute_angle(a, b, c)

1.34 s ± 28.4 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)


<TimeitResult : 1.34 s ± 28.4 ms per loop (mean ± std. dev. of 5 runs, 2 loops each)>

In [5]:
# store the timing result
GPU_TIMING = _

### Numpy Version
Notice the numpy version of the same calculation looks nearly identical to the Ufunc definition.  

In [6]:
%%timeit -n1 -r1 -o
# CPU version
C_CPU = np.arccos(( a**2 + b**2 - c**2 ) / ( 2.0 * a * b ))

5.78 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 5.78 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [7]:
# store the timing result
CPU_TIMING = _

### Computing Speedup Factor

In [8]:
print('Speedup factor: ', CPU_TIMING.average / GPU_TIMING.average, 'X')

Speedup factor:  4.324815961466581 X


### Checking Results

In [9]:
# recompute (workaround for timeit bug)
C_GPU = compute_angle(a, b, c)
C_CPU = np.arccos(( a**2 + b**2 - c**2 ) / ( 2.0 * a * b ))

tol=1e-5
if np.array(np.abs(C_CPU-C_GPU)<tol).sum()==N:
    print('results agree')

results agree
